## Estimating LDA with Bank of England Minutes

Here we show the code to re-create figures in http://bit.ly/2G7qoF3 using latent Dirichlet allocation.  First, we load and process MPC minutes exactly as in the information_retrieval notebook.  The reason for repeating the steps is that we treat a document as a single paragraph of the minutes rather than an entire minutes file.

A similar notebook (with somewhat more extensive explanation) on state-of-the-union addresses can be found in the text-mining-tutorial repository.

In [1]:
import pandas as pd
import numpy as np
import topicmodels

data = pd.read_table("mpc_minutes.txt", encoding="utf-8")

docsobj = topicmodels.RawDocs(data.minutes, "long")
docsobj.token_clean(1)
docsobj.stopword_remove("tokens")
docsobj.stem()
docsobj.stopword_remove("stems")
docsobj.term_rank("stems")

The first step in estimation is to initialize a model. We will pass docsobj.stems as the set of documents, and we also need to decide on a number of topics. Here we choose 30 topics.

In [2]:
ldaobj = topicmodels.LDA.LDAGibbs(docsobj.stems,30)

There are three main parameters in LDA, the number of topics, and the two hyperparameters of the Dirichlet priors.  We follow the advice of Griffiths and Steyvers (2004) and sets the hyperparameter of the Dirichlet prior on topics to $200/V$, where $V$ is the number of unique vocabulary elements, and the hyperparameter of the Dirichlet prior on document-topic distributions to $50/K$, where $K$ is the number of topics.

In [3]:
print(ldaobj.K) # number of topic, user defined.
print(ldaobj.alpha) # hyperparameter for document-topic distribution, automatically defined
print(ldaobj.beta) # hyperparameter for topics, automatically defined

30
1.6666666666666667
0.03622532149972831


Now that we have initialized our topic model, we are ready to sample. To sample, we pass three parameters. The first is the number of iterations we want the chain to burn in before beginning to sample. The second is a thinning interval, the number of iterations to let the chain run between samples. Allowing for a thinning interval reduces autocorrelation between samples. The third is the number of samples to take. So, for example, if the user passes (2000,50,20) the following will happen. First, the chain will run for 2,000 iterations. Then 20 samples will be taken corresponding to the $\{2050,2100\ldots,2950,3000\}$ iterations for a total of 3000 iterations overall.

In [4]:
ldaobj.sample(2000, 50, 20)

Iteration 10 of (collapsed) Gibbs sampling
Iteration 20 of (collapsed) Gibbs sampling
Iteration 30 of (collapsed) Gibbs sampling
Iteration 40 of (collapsed) Gibbs sampling
Iteration 50 of (collapsed) Gibbs sampling
Iteration 60 of (collapsed) Gibbs sampling
Iteration 70 of (collapsed) Gibbs sampling
Iteration 80 of (collapsed) Gibbs sampling
Iteration 90 of (collapsed) Gibbs sampling
Iteration 100 of (collapsed) Gibbs sampling
Iteration 110 of (collapsed) Gibbs sampling
Iteration 120 of (collapsed) Gibbs sampling
Iteration 130 of (collapsed) Gibbs sampling
Iteration 140 of (collapsed) Gibbs sampling
Iteration 150 of (collapsed) Gibbs sampling
Iteration 160 of (collapsed) Gibbs sampling
Iteration 170 of (collapsed) Gibbs sampling
Iteration 180 of (collapsed) Gibbs sampling
Iteration 190 of (collapsed) Gibbs sampling
Iteration 200 of (collapsed) Gibbs sampling
Iteration 210 of (collapsed) Gibbs sampling
Iteration 220 of (collapsed) Gibbs sampling
Iteration 230 of (collapsed) Gibbs sampli

Iteration 1860 of (collapsed) Gibbs sampling
Iteration 1870 of (collapsed) Gibbs sampling
Iteration 1880 of (collapsed) Gibbs sampling
Iteration 1890 of (collapsed) Gibbs sampling
Iteration 1900 of (collapsed) Gibbs sampling
Iteration 1910 of (collapsed) Gibbs sampling
Iteration 1920 of (collapsed) Gibbs sampling
Iteration 1930 of (collapsed) Gibbs sampling
Iteration 1940 of (collapsed) Gibbs sampling
Iteration 1950 of (collapsed) Gibbs sampling
Iteration 1960 of (collapsed) Gibbs sampling
Iteration 1970 of (collapsed) Gibbs sampling
Iteration 1980 of (collapsed) Gibbs sampling
Iteration 1990 of (collapsed) Gibbs sampling
Iteration 2000 of (collapsed) Gibbs sampling
Iteration 2010 of (collapsed) Gibbs sampling
Iteration 2020 of (collapsed) Gibbs sampling
Iteration 2030 of (collapsed) Gibbs sampling
Iteration 2040 of (collapsed) Gibbs sampling
Iteration 2050 of (collapsed) Gibbs sampling
Iteration 2060 of (collapsed) Gibbs sampling
Iteration 2070 of (collapsed) Gibbs sampling
Iteration 

One simple convergence criterion is whether the perplexity computed at each draw of the Markov chain is stable, which we can compute and verify.

In [5]:
ldaobj.perplexity()

array([435.47314257, 435.5966975 , 435.68398388, 435.40008065,
       435.40899925, 435.53247625, 435.72287498, 435.50826588,
       435.28085581, 435.40925086, 435.61699448, 435.4973021 ,
       435.51415415, 435.5234896 , 435.47166945, 435.09234773,
       435.27532876, 435.39617475, 435.4746255 , 435.49317599])

The two key outputs of interest are the posterior distributions for the $\boldsymbol\beta$ topic-term probability vectors and for the $\boldsymbol\theta$ document-topic shares.  We can compute these for each draw of the Markov chain, which are stored in the objects below.

In [6]:
print(ldaobj.tt.shape)
print(ldaobj.dt.shape)

(5521, 30, 20)
(7277, 30, 20)


We can first explore the terms that make up each topic via the call below, which outputs a file with terms in each topic ranked by their average probability across the samples...the user can explore the contents offline.

In [7]:
ldaobj.topic_content(20, output_file="topic_description.csv")

For the estimation of LDA, we used individual paragraphs as documents, but most analysis we would naturally be more interested in the distribution of topics at the level of the entire minutes.  To obtain these, we re-estimate topic shares for aggregated documents, holding fixed the estimated $\boldsymbol\beta$ vectors from the LDA estimation above.  The code below outputs a file with the relevant output.

In [8]:
data['temp'] = [' '.join(s) for s in docsobj.stems]
aggspeeches = data.groupby(['year'])['temp'].apply(lambda x: ' '.join(x))
aggdocs = topicmodels.RawDocs(aggspeeches)

queryobj = topicmodels.LDA.QueryGibbs(aggdocs.tokens,
                                      ldaobj.token_key,
                                      ldaobj.tt)
queryobj.query(10)

dt_query = queryobj.dt_avg()
aggdata = pd.DataFrame(dt_query, index=aggspeeches.index,
                       columns=['T' + str(i) for i in range(queryobj.K)])
aggdata.to_csv("final_output_agg.csv")

Sample 0 queried
Sample 1 queried
Sample 2 queried
Sample 3 queried
Sample 4 queried
Sample 5 queried
Sample 6 queried
Sample 7 queried
Sample 8 queried
Sample 9 queried
Sample 10 queried
Sample 11 queried
Sample 12 queried
Sample 13 queried
Sample 14 queried
Sample 15 queried
Sample 16 queried
Sample 17 queried
Sample 18 queried
Sample 19 queried


Try to verify whether you can identify a topic about "financial markets" and explore its time series variation.  Does this resemble the figures in the speech linked to above?